In [1]:
import quandl as ql
import numpy as np
import pandas as pd 

In [12]:
def get_wti_gasoline_frame() -> pd.DataFrame:
    # wti - gas
    # $barrel - $gallon
#     petro_prices = ql.get(["CHRIS/ICE_T1"])
#     petro_futures_df = petro_prices.loc[:, ["CHRIS/ICE_T1 - Settle"]]
#     petro_futures_df.columns = ['wti']
#     petro_futures_df.index = pd.to_datetime(petro_futures_df.index) 
    
    gas_contr = pd.read_csv('./data/nymex_gasoline_future_contract.csv')
    gas_contr['date'] = pd.to_datetime(gas_contr['date'])
    gas_contr.set_index('date', inplace=True)
    gas_contr.columns = ['gasoline']
    
    wti_contr = pd.read_csv('./data/nymex_wti_future_contract.csv')
    wti_contr['date'] = pd.to_datetime(wti_contr['date'])
    wti_contr.set_index('date', inplace=True)
    wti_contr.columns = ['wti']
    
    futures_df = pd.concat([wti_contr, gas_contr], axis=1)

    # Convert $/gallon to $/barrel; 1 barrel = 42 gallons
    futures_df['gasoline'] = futures_df['gasoline']*42

    return combine_legs_and_calc_spread(futures_df)

In [8]:
def get_wti_brent_frame() -> pd.DataFrame:
    oil_prices = ql.get(["CHRIS/ICE_T1", "CHRIS/ICE_B1"])
    oil_futures_df = oil_prices.loc[:, ["CHRIS/ICE_T1 - Settle", "CHRIS/ICE_B1 - Settle"]]
    oil_futures_df.columns = ['wti', 'brent']
    
    return combine_legs_and_calc_spread(oil_futures_df)

In [ ]:
def get_soybean_soyoil_frame() -> pd.DataFrame:

    soy_futures_prices = ql.get(["CHRIS/CME_S1", "CHRIS/CME_BO1"]).last('20Y')

    futures_df = soy_futures_prices.loc[:, ["CHRIS/CME_S1 - Last", "CHRIS/CME_BO1 - Last"]]
    futures_df.columns = ['soybean', 'soyoil']
    futures_df.loc[:, 'soyoil'] = futures_df.loc[:, 'soyoil']  * 11
    
    return combine_legs_and_calc_spread(futures_df)

In [ ]:
def get_ethanol_corn_frame() -> pd.DataFrame:
    eth_futures_prices = ql.get(["CHRIS/CME_EH1", "CHRIS/CME_C1"])

    futures_df = eth_futures_prices.loc[:, ["CHRIS/CME_EH1 - Last", "CHRIS/CME_C1 - Last"]]
    futures_df.columns = ['ethanol', 'corn']
    futures_df.loc[:, 'ethanol'] =  ( 2.8 * futures_df['ethanol'] ) * 100
    
    return combine_legs_and_calc_spread(futures_df)

In [10]:
def combine_legs_and_calc_spread(df: pd.DataFrame):
    """
    This is calculating the spread the Dunis way, which simply means
    neglecting the cointegration factor to enforce a money neutral poisiton
    and investing the same amount in each leg.
    """

    new_df = df.copy().ffill()
    # Spread in $/metric (gallon, barrel, bushel)
    new_df['dollar_spread'] = (df.iloc[:, 0]) - (df.iloc[:, 1])
    
    # Returns the Dunis Way
    leg1_rets = (df.iloc[:, 0] - df.iloc[:, 0].shift(1)) / df.iloc[:, 0].shift(1)
    leg2_rets = (df.iloc[:, 1] - df.iloc[:, 1].shift(1)) / df.iloc[:, 1].shift(1)

    new_df[df.columns[0] + '_rets'] = leg1_rets
    new_df[df.columns[1] + '_rets'] = leg2_rets
    new_df['spread_rets'] = leg1_rets - leg2_rets
#     new_df['log_spread'] = np.log(df.iloc[:, 0]) - np.log(df.iloc[:, 1])
#     new_df['log_spread_ret'] = new_df['log_spread'].diff() 
    return new_df.dropna()

In [13]:
get_wti_gasoline_frame().to_csv('./data/wti_gas.csv')

In [15]:
get_wti_brent_frame().to_csv('./data/wti_brent.csv')

In [3]:
import yfinance as yf

yf.download(['RBZ21.NYM'])

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-24,1.3750,1.3750,1.3488,1.3859,1.3859,35
2020-01-26,1.3544,1.3544,1.3544,1.3544,1.3544,0
2020-01-27,1.3293,1.3293,1.3290,1.3544,1.3544,40
2020-01-28,1.3492,1.3492,1.3492,1.3392,1.3392,16
2020-01-29,1.3550,1.3560,1.3550,1.3485,1.3485,38
...,...,...,...,...,...,...
2020-11-16,1.1825,1.1942,1.1751,1.1783,1.1783,1752
2020-11-17,1.1765,1.1829,1.1617,1.1783,1.1783,1508
2020-11-18,1.1921,1.1988,1.1864,1.1869,1.1869,1858
